In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.simplefilter('ignore')
init_notebook_mode(connected = True)
tqdm.pandas()

In [2]:
df = pd.read_csv('train_data.csv')
df.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [3]:
call = pd.read_csv('call_data.csv', usecols=['order_id', 'reason_text', 'user_type', 'rider_id'])
call.head()

,order_id,reason_text,user_type,rider_id
0,556753,NaN,customer,11696
1,556753,NaN,customer,11696
2,556754,NaN,customer,18117
3,556755,NaN,customer,18623
4,556755,NaN,customer,18623


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_id              450000 non-null  int64  
 2   order_date            450000 non-null  object 
 3   allot_time            450000 non-null  object 
 4   accept_time           449843 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              450000 non-null  int64  
 8   first_mile_distance   450000 non-null  float64
 9   last_mile_distance    450000 non-null  float64
 10  alloted_orders        433052 non-null  float64
 11  delivered_orders      432659 non-null  float64
 12  cancelled             450000 non-null  int64  
 13  undelivered_orders    432659 non-null  float64
 14  lifetime_order_count  449947 non-null  float64
 15  

In [5]:
call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562624 entries, 0 to 562623
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   order_id     562624 non-null  int64 
 1   reason_text  37061 non-null   object
 2   user_type    562624 non-null  object
 3   rider_id     562624 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 17.2+ MB


In [6]:
call[~call['reason_text'].isnull()].head()

,order_id,reason_text,user_type,rider_id
14,556773,Items Not Available at Restaurant,support,12885
15,556773,Items Not Available at Restaurant,support,12885
16,556773,Restaurant Closed,support,12885
17,556773,Items Not Available at Restaurant,support,12885
18,556773,Restaurant Closed,support,12885


In [7]:
call[call['order_id'] == 556755]

,order_id,reason_text,user_type,rider_id
3,556755,NaN,customer,18623
4,556755,NaN,customer,18623


In [8]:
call['reason_text'].value_counts()

Others                                                    11536
Customer not responding to calls                          11178
Items Not Available at Restaurant                          5439
Customer requesting to deliver at a different location     3574
Cancel order due to bike issue                             2154
Restaurant Closed                                          1903
Long distance order                                         841
Cancel order due to heavy traffic                           286
Cancel order due to bad weather                             150
Name: reason_text, dtype: int64

In [9]:
df[df['order_id'] == 556755]

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.8,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN


In [10]:
df['reassigned_order'].value_counts()

1.0    13753
Name: reassigned_order, dtype: int64

In [11]:
df['reassignment_method'].value_counts()

auto      13383
manual      361
Name: reassignment_method, dtype: int64

In [12]:
df['reassignment_reason'].value_counts()

Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket    7212
Reassignment Request from SE portal.                                            5300
Reassign                                                                        1241
Name: reassignment_reason, dtype: int64

In [13]:
df[~df['reassigned_order'].isnull()].head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
9,2021-01-26 03:15:18,556762,2021-01-26 00:00:00,2021-01-26 03:21:27,2021-01-26 03:22:04,2021-01-26 04:14:56,2021-01-26 04:38:39,21037,2.8786,2.61,NaN,NaN,0,NaN,1.0,auto,Reassignment Request from SE portal.,1.0,NaN,NaN
20,2021-01-26 03:28:13,556773,2021-01-26 00:00:00,2021-01-26 04:09:50,2021-01-26 04:11:06,NaN,NaN,12885,2.8438,6.81,29.0,29.0,1,0.0,449.0,auto,Reassignment Request from SE portal.,1.0,NaN,2021-01-26 04:51:46
38,2021-01-26 03:38:14,556791,2021-01-26 00:00:00,2021-01-26 03:54:07,2021-01-26 03:54:28,2021-01-26 04:09:45,2021-01-26 04:15:23,10869,1.5504,0.17,145.0,144.0,0,1.0,2037.0,auto,Auto Reassignment basis Inaction. coreengine.t...,1.0,56.683333,NaN
52,2021-01-26 03:42:17,556805,2021-01-26 00:00:00,2021-01-26 03:58:13,2021-01-26 03:59:35,2021-01-26 04:13:25,2021-01-26 04:44:06,313,2.9776,1.43,76.0,76.0,0,0.0,439.0,auto,Auto Reassignment basis Inaction. coreengine.t...,1.0,NaN,NaN
100,2021-01-26 03:54:11,556853,2021-01-26 00:00:00,2021-01-26 05:29:06,2021-01-26 05:30:01,NaN,NaN,2391,2.9217,5.20,118.0,114.0,1,4.0,860.0,auto,Reassignment Request from SE portal.,1.0,22.516667,2021-01-26 05:45:51


In [14]:
df[df['rider_id']==1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
315699,2021-02-03 06:43:10,239802,2021-02-03 00:00:00,2021-02-03 06:43:25,2021-02-03 06:43:30,2021-02-03 06:55:20,2021-02-03 07:12:18,1,1.4561,4.14,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,147.533333,NaN
328569,2021-02-03 13:38:38,252675,2021-02-03 00:00:00,2021-02-03 13:38:55,2021-02-03 13:40:17,2021-02-03 14:06:26,2021-02-03 14:46:21,1,2.4127,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,510.566667,NaN
331400,2021-02-03 14:02:48,255506,2021-02-03 00:00:00,2021-02-03 14:04:44,2021-02-03 14:05:50,2021-02-03 14:15:04,2021-02-03 14:33:03,1,0.8029,3.68,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,534.733333,NaN
346038,2021-02-03 16:14:20,270142,2021-02-03 00:00:00,2021-02-03 16:14:55,2021-02-03 16:15:50,2021-02-03 16:31:13,2021-02-03 16:40:45,1,1.1215,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,666.266667,NaN
348876,2021-02-03 16:41:11,272980,2021-02-03 00:00:00,2021-02-03 16:41:47,2021-02-03 16:42:43,2021-02-03 16:49:22,2021-02-03 16:58:00,1,0.4628,1.51,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,693.116667,NaN
352353,2021-02-03 17:14:48,276457,2021-02-03 00:00:00,2021-02-03 17:15:39,2021-02-03 17:16:26,2021-02-03 17:28:24,2021-02-03 17:35:16,1,1.1628,1.36,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,726.733333,NaN
353553,2021-02-03 17:26:31,277657,2021-02-03 00:00:00,2021-02-03 17:26:55,2021-02-03 17:27:51,2021-02-03 17:41:59,2021-02-03 18:04:50,1,1.0028,3.59,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,738.450000,NaN
373828,2021-02-04 14:14:41,216800,2021-02-04 00:00:00,2021-02-04 14:15:41,2021-02-04 14:16:31,2021-02-04 14:29:13,2021-02-04 14:41:52,1,0.8283,1.91,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,519.200000,NaN
379722,2021-02-04 15:10:50,222695,2021-02-04 00:00:00,2021-02-04 15:11:47,2021-02-04 15:12:24,2021-02-04 15:15:29,2021-02-04 15:23:42,1,0.2979,0.58,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,575.350000,NaN
383660,2021-02-04 15:48:53,226632,2021-02-04 00:00:00,2021-02-04 15:49:39,2021-02-04 15:49:52,2021-02-04 15:55:55,2021-02-04 16:04:07,1,0.8439,0.62,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,613.400000,NaN


In [15]:
call['user_type'].value_counts()

customer    525563
support      37061
Name: user_type, dtype: int64

- Analysing which Day in a week has the max ratio of orders cancelled to placed
- **Most frequent reason given by a rider** (DONE)
- Relation between the time till cancellation and the type of call made
- **Lifetime Order Count wrong values** (DONE)
- Time of the day when you have highest cancelled:orders placed ratio
- **How cancellation increases with trip distances** (DONE)
- New vs Medium vs experienced rider cancellation rate
- **Cancellation vs number of calls** (DONE)
- For orders cancelled after food prep, time taken to cancel from pickup time
- **Null Reason analysis based on cancellation time** 
- Cancellation Rate vs Average Session Time per day

In [158]:
def get_hist(col, title = ''):
    trace = go.Histogram(x = col, nbinsx = 200, histnorm='percent')
    data = [trace]
    if title == '':
        title = col.name
    layout = go.Layout(title = title)
    
    fig = go.Figure(data = data, layout = layout)
    
    iplot(fig)

## Most frequent reason given by a rider

In [17]:
freq_reason = call[~call['reason_text'].isnull()]

In [18]:
freq_reason = freq_reason[['rider_id', 'reason_text', 'order_id']].groupby(['rider_id', 'reason_text']).agg('count').reset_index()
freq_reason.columns = ['rider_id', 'reason_text', 'count']
freq_reason.head()

,rider_id,reason_text,count
0,0,Customer not responding to calls,4
1,1,Customer not responding to calls,2
2,1,Others,2
3,1,Restaurant Closed,1
4,3,Restaurant Closed,2


In [19]:
freq_reason = freq_reason.loc[freq_reason.groupby(['rider_id'])['count'].idxmax()]

In [20]:
freq_reason.head()

,rider_id,reason_text,count
0,0,Customer not responding to calls,4
1,1,Customer not responding to calls,2
4,3,Restaurant Closed,2
5,6,Customer not responding to calls,4
7,9,Customer not responding to calls,1


In [159]:
get_hist(freq_reason['reason_text'], title = 'Percentage Distribution of Reason for Cancellation')

## Lifetime Order Count Wrong Values

In [22]:
df[df['rider_id']==1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
315699,2021-02-03 06:43:10,239802,2021-02-03 00:00:00,2021-02-03 06:43:25,2021-02-03 06:43:30,2021-02-03 06:55:20,2021-02-03 07:12:18,1,1.4561,4.14,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,147.533333,NaN
328569,2021-02-03 13:38:38,252675,2021-02-03 00:00:00,2021-02-03 13:38:55,2021-02-03 13:40:17,2021-02-03 14:06:26,2021-02-03 14:46:21,1,2.4127,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,510.566667,NaN
331400,2021-02-03 14:02:48,255506,2021-02-03 00:00:00,2021-02-03 14:04:44,2021-02-03 14:05:50,2021-02-03 14:15:04,2021-02-03 14:33:03,1,0.8029,3.68,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,534.733333,NaN
346038,2021-02-03 16:14:20,270142,2021-02-03 00:00:00,2021-02-03 16:14:55,2021-02-03 16:15:50,2021-02-03 16:31:13,2021-02-03 16:40:45,1,1.1215,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,666.266667,NaN
348876,2021-02-03 16:41:11,272980,2021-02-03 00:00:00,2021-02-03 16:41:47,2021-02-03 16:42:43,2021-02-03 16:49:22,2021-02-03 16:58:00,1,0.4628,1.51,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,693.116667,NaN
352353,2021-02-03 17:14:48,276457,2021-02-03 00:00:00,2021-02-03 17:15:39,2021-02-03 17:16:26,2021-02-03 17:28:24,2021-02-03 17:35:16,1,1.1628,1.36,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,726.733333,NaN
353553,2021-02-03 17:26:31,277657,2021-02-03 00:00:00,2021-02-03 17:26:55,2021-02-03 17:27:51,2021-02-03 17:41:59,2021-02-03 18:04:50,1,1.0028,3.59,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,738.450000,NaN
373828,2021-02-04 14:14:41,216800,2021-02-04 00:00:00,2021-02-04 14:15:41,2021-02-04 14:16:31,2021-02-04 14:29:13,2021-02-04 14:41:52,1,0.8283,1.91,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,519.200000,NaN
379722,2021-02-04 15:10:50,222695,2021-02-04 00:00:00,2021-02-04 15:11:47,2021-02-04 15:12:24,2021-02-04 15:15:29,2021-02-04 15:23:42,1,0.2979,0.58,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,575.350000,NaN
383660,2021-02-04 15:48:53,226632,2021-02-04 00:00:00,2021-02-04 15:49:39,2021-02-04 15:49:52,2021-02-04 15:55:55,2021-02-04 16:04:07,1,0.8439,0.62,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,613.400000,NaN


In [23]:
riders = df[['rider_id', 'order_date', 'lifetime_order_count']].groupby(['rider_id', 'order_date']).count().reset_index()
riders.head()

,rider_id,order_date,lifetime_order_count
0,0,2021-01-27 00:00:00,1
1,0,2021-01-29 00:00:00,2
2,0,2021-02-01 00:00:00,3
3,0,2021-02-04 00:00:00,1
4,1,2021-02-03 00:00:00,7


In [65]:
master = pd.DataFrame()

prev_rider = ''
prev_date = ''
prev_lifetime = 0
inc = 0
for i, row in tqdm(riders.iterrows(), total = len(riders)):
    temp  = df[df['rider_id'] == row['rider_id']][df['order_date'] == row['order_date']]
    if row['order_date'] != prev_date:
        inc += prev_lifetime
        prev_lifetime = row['lifetime_order_count']
    if row['rider_id'] != prev_rider:
        inc = 0
    temp['lifetime_order_count'] += inc

    prev_date = row['order_date']
    prev_rider = row['rider_id']
    master = master.append(temp)

master.to_csv('train_mod.csv', index = False)

100%|███████████████████████████████████████████████████████| 94620/94620 [1:48:25<00:00, 14.55it/s]


In [24]:
df = pd.read_csv('train_mod.csv')
df.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,29.0,28.0,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN
1,2021-01-29 08:37:58,442759,2021-01-29 00:00:00,2021-01-29 08:37:58,2021-01-29 08:39:57,2021-01-29 08:57:45,2021-01-29 09:00:23,0,1.3477,0.03,28.0,28.0,0,0.0,8735.0,NaN,NaN,NaN,245.266667,NaN
2,2021-01-29 08:38:07,442767,2021-01-29 00:00:00,2021-01-29 09:35:08,2021-01-29 09:36:58,NaN,NaN,0,1.6314,2.22,28.0,28.0,1,0.0,8735.0,auto,Reassign,1.0,245.416667,2021-01-29 09:54:21
3,2021-02-01 14:31:33,339093,2021-02-01 00:00:00,2021-02-01 14:31:34,2021-02-01 14:33:32,2021-02-01 14:53:07,2021-02-01 15:00:36,0,0.5356,1.01,26.0,25.0,0,1.0,8737.0,NaN,NaN,NaN,646.016667,NaN
4,2021-02-01 14:54:31,341573,2021-02-01 00:00:00,2021-02-01 15:00:55,2021-02-01 15:01:01,2021-02-01 15:08:47,2021-02-01 15:16:06,0,2.5956,1.56,26.0,25.0,0,1.0,8737.0,auto,Reassignment Request from SE portal.,1.0,668.983333,NaN


In [26]:
max_rider = df.groupby(['rider_id']).agg({'order_id':'count'}).reset_index()
max_rider.columns = ['rider_id', 'order_count']
max_rider.head()

,rider_id,order_count
0,0,7
1,1,16
2,2,28
3,4,3
4,5,19


In [43]:
top_3 = max_rider[max_rider['order_count']>10].sort_values('order_count').iloc[:15, :]
top_3

,rider_id,order_count
16177,18207,11
8673,10122,11
5683,6554,11
12491,14521,11
5680,6551,11
12480,14510,11
5653,6523,11
921,1045,11
12479,14509,11
8685,10135,11


In [107]:
top_3_riders = [237, 190, 11998]

for rider in top_3_riders:
    lifetime = df[df['rider_id'] == rider]
    lifetime = lifetime.groupby('order_date').agg('first').reset_index()
    fig = px.line(lifetime, x = 'order_date', y = 'lifetime_order_count', title = 'Lifetime Order Count for Rider ID: '+ str(rider))
    fig.show()

In [108]:
mid_3_riders = [7141, 12492, 6523]

for rider in mid_3_riders:
    lifetime = df[df['rider_id'] == rider]
    lifetime = lifetime.groupby('order_date').agg('first').reset_index()
    fig = px.line(lifetime, x = 'order_date', y = 'lifetime_order_count', title = 'Lifetime Order Count for Rider ID: '+ str(rider))
    fig.show()

###  How Cancellation increases with Trip Distance

In [133]:
canc_dist = df

In [134]:
canc_dist.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,29.0,28.0,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN
1,2021-01-29 08:37:58,442759,2021-01-29 00:00:00,2021-01-29 08:37:58,2021-01-29 08:39:57,2021-01-29 08:57:45,2021-01-29 09:00:23,0,1.3477,0.03,28.0,28.0,0,0.0,8735.0,NaN,NaN,NaN,245.266667,NaN
2,2021-01-29 08:38:07,442767,2021-01-29 00:00:00,2021-01-29 09:35:08,2021-01-29 09:36:58,NaN,NaN,0,1.6314,2.22,28.0,28.0,1,0.0,8735.0,auto,Reassign,1.0,245.416667,2021-01-29 09:54:21
3,2021-02-01 14:31:33,339093,2021-02-01 00:00:00,2021-02-01 14:31:34,2021-02-01 14:33:32,2021-02-01 14:53:07,2021-02-01 15:00:36,0,0.5356,1.01,26.0,25.0,0,1.0,8737.0,NaN,NaN,NaN,646.016667,NaN
4,2021-02-01 14:54:31,341573,2021-02-01 00:00:00,2021-02-01 15:00:55,2021-02-01 15:01:01,2021-02-01 15:08:47,2021-02-01 15:16:06,0,2.5956,1.56,26.0,25.0,0,1.0,8737.0,auto,Reassignment Request from SE portal.,1.0,668.983333,NaN


In [135]:
canc_dist['first_mile_distance'] = pd.cut(canc_dist['first_mile_distance'], 10)
canc_dist['last_mile_distance'] = pd.cut(canc_dist['last_mile_distance'], 10)
canc_dist.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,"(-0.0419, 4.204]","(-0.0224, 2.241]",29.0,28.0,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN
1,2021-01-29 08:37:58,442759,2021-01-29 00:00:00,2021-01-29 08:37:58,2021-01-29 08:39:57,2021-01-29 08:57:45,2021-01-29 09:00:23,0,"(-0.0419, 4.204]","(-0.0224, 2.241]",28.0,28.0,0,0.0,8735.0,NaN,NaN,NaN,245.266667,NaN
2,2021-01-29 08:38:07,442767,2021-01-29 00:00:00,2021-01-29 09:35:08,2021-01-29 09:36:58,NaN,NaN,0,"(-0.0419, 4.204]","(-0.0224, 2.241]",28.0,28.0,1,0.0,8735.0,auto,Reassign,1.0,245.416667,2021-01-29 09:54:21
3,2021-02-01 14:31:33,339093,2021-02-01 00:00:00,2021-02-01 14:31:34,2021-02-01 14:33:32,2021-02-01 14:53:07,2021-02-01 15:00:36,0,"(-0.0419, 4.204]","(-0.0224, 2.241]",26.0,25.0,0,1.0,8737.0,NaN,NaN,NaN,646.016667,NaN
4,2021-02-01 14:54:31,341573,2021-02-01 00:00:00,2021-02-01 15:00:55,2021-02-01 15:01:01,2021-02-01 15:08:47,2021-02-01 15:16:06,0,"(-0.0419, 4.204]","(-0.0224, 2.241]",26.0,25.0,0,1.0,8737.0,auto,Reassignment Request from SE portal.,1.0,668.983333,NaN


In [138]:
canc_dist['first_mile_distance'].unique()

[(-0.0419, 4.204], (4.204, 8.408], (8.408, 12.612], (37.834, 42.038], (16.815, 21.019]]
Categories (10, interval[float64, right]): [(-0.0419, 4.204] < (4.204, 8.408] < (8.408, 12.612] < (12.612, 16.815] ... (25.223, 29.427] < (29.427, 33.631] < (33.631, 37.834] < (37.834, 42.038]]

In [136]:
def get_interval_mean(val):
    return (val.left + val.right)/2

In [152]:
uniques = canc_dist['last_mile_distance'].unique()
print(len(uniques))
vals = pd.DataFrame()
for fmd in uniques:
    temp = canc_dist[canc_dist['last_mile_distance'] == fmd]
    vals = vals.append(pd.Series([fmd,sum(temp['cancelled'])/len(temp['cancelled'])]), ignore_index=True)
vals.columns = ['last_mile_distance', 'Cancel']
vals['last_mile_distance'] = vals['last_mile_distance'].apply(get_interval_mean)
vals

10


,last_mile_distance,Cancel
0,1.1093,0.010217
1,3.3615,0.011220
2,5.6025,0.014398
3,7.8435,0.016422
4,10.0845,0.020249
5,12.3255,0.039394
6,14.5665,0.093023
7,21.2895,0.000000
8,16.8075,0.200000
9,19.0485,0.000000


In [157]:
fig = px.bar(vals, x = 'last_mile_distance', y = 'Cancel', title = 'Normalised Cancellation Rate vs Delivery Distance')
fig.show()

In [130]:
last_count = canc_dist.groupby('last_mile_distance').agg({'order_id': 'count'}).reset_index()
last_count.columns = ['last_mile_distance', 'count']
last_count['last_mile_distance'] = last_count['last_mile_distance'].apply(get_interval_mean)
last_count.head()

,last_mile_distance,count
0,0.85485,1382
1,2.59050,1550
2,4.31750,1345
3,6.04450,693
4,7.77150,165


In [132]:
fig = px.bar(last_count, x = 'last_mile_distance', y = 'count')
fig.show()

In [103]:
get_hist(canc_dist['first_mile_distance'], title = 'Cancellation Variation With Pickup Distance')

In [104]:
get_hist(canc_dist['last_mile_distance'], title = 'Cancellation Variation With Drop Distance')

## Cancellation vs Number of Calls

In [47]:
joint = pd.merge(df, call.drop('rider_id', axis = 1), on='order_id', how = 'left')
joint.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,...,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,reason_text,user_type
0,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN,NaN,customer
1,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN,Customer not responding to calls,support
2,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN,NaN,customer
3,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN,NaN,customer
4,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,0,1.0,8734.0,NaN,NaN,NaN,299.583333,NaN,NaN,customer


In [60]:
num_calls = joint.groupby(['order_id']).agg({'order_date': 'count', 'cancelled':'first'}).reset_index()
num_calls.head()

,order_id,order_date,cancelled
0,118350,1,0
1,118351,1,0
2,118352,1,0
3,118353,1,0
4,118354,1,0


In [61]:
num_calls = num_calls.groupby(['cancelled']).agg({'order_date':'mean'}).reset_index()

In [62]:
num_calls

,cancelled,order_date
0,0,1.280724
1,1,2.290205


In [63]:
num_calls.columns = ['Cancelled', 'Mean Number of Calls Made']

In [160]:
fig = px.bar(num_calls, x = 'Cancelled', y = 'Mean Number of Calls Made', title = 'Mean Number of Calls for Not Cancelled vs Cancelled')
fig.show()

## Null Reason Analysis based on Cancellation Time

In [65]:
def time_diff(time_list_1, time_list_2):
    hour_diff = (int(time_list_1[0]) - int(time_list_2[0]))*3600
    min_diff = (int(time_list_1[1]) - int(time_list_2[1]))*60
    sec_diff = (int(time_list_1[2]) - int(time_list_2[2]))
    return hour_diff + min_diff + sec_diff

In [76]:
def make_cancellation_time_feature(row):
    if (row['cancelled_time'] == row['cancelled_time']) and (row['pickup_time'] == row['pickup_time']):
        cancel_list = row['cancelled_time'].split(' ')[1].split(':')
        pickup_time = row['pickup_time'].split(' ')[1].split(':')
        
        time = time_diff(cancel_list, pickup_time)
        if time<0:
            time += 3600*24
        row['time_to_cancel'] = time
    else:
        row['time_to_cancel'] = np.nan
        
    return row

cancel = df.progress_apply(make_cancellation_time_feature, axis = 1)

100%|█████████████████████████████████████████████████████| 450000/450000 [02:43<00:00, 2747.38it/s]


In [77]:
cancel = cancel[~cancel['time_to_cancel'].isnull()]
cancel.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,...,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,time_to_cancel
84,2021-02-03 06:41:09,239679,2021-02-03 00:00:00,2021-02-03 06:41:57,2021-02-03 06:42:03,2021-02-03 06:46:19,NaN,6,0.3369,4.56,...,51.0,1,0.0,157.0,NaN,NaN,NaN,11.150000,2021-02-03 07:26:51,2432.0
206,2021-02-05 13:46:16,172227,2021-02-05 00:00:00,2021-02-05 13:46:57,2021-02-05 13:47:01,2021-02-05 14:00:54,NaN,10,2.4971,0.02,...,202.0,1,0.0,9788.0,NaN,NaN,NaN,654.550000,2021-02-05 14:02:43,109.0
225,2021-01-30 07:20:33,401533,2021-01-30 00:00:00,2021-01-30 07:20:54,2021-01-30 07:21:35,2021-01-30 07:38:21,NaN,11,0.6486,2.33,...,37.0,1,0.0,311.0,NaN,NaN,NaN,189.116667,2021-01-30 08:07:30,1749.0
505,2021-02-03 13:41:00,252900,2021-02-03 00:00:00,2021-02-03 13:41:40,2021-02-03 13:42:33,2021-02-03 13:48:10,NaN,19,0.3676,3.01,...,23.0,1,0.0,703.0,NaN,NaN,NaN,370.216667,2021-02-03 15:05:41,4651.0
569,2021-02-02 14:37:21,300409,2021-02-02 00:00:00,2021-02-02 14:39:51,2021-02-02 14:40:11,2021-02-02 15:01:40,NaN,20,0.6850,1.75,...,180.0,1,1.0,1381.0,NaN,NaN,NaN,457.600000,2021-02-02 15:26:47,1507.0


In [80]:
get_hist(cancel['time_to_cancel'], title = 'Time taken by Rider to Cancel in seconds')